<a href="https://colab.research.google.com/github/junghodavidlee/Project-3/blob/maddie_jankowski/Project_3_MJ_final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install fastparquet
!pip install gradio
!pip install vaderSentiment
!pip install nltk
!pip install datasets

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 362, in run
    resolver = self.make_resolver(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 177, in make_resolver
    return pip._internal.resolution.resolvelib.resolver.Resolver(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 58, in __init__
    self.factory = Factory(
                   ^^^^^^^^
  File "/usr/local/lib/py

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import gradio as gr
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import joblib
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.svm import LinearSVC


# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

In [ ]:
# Import the IMDB dataset
from google.colab import files
uploaded = files.upload()


Saving IMDB Dataset.csv to IMDB Dataset (1).csv


In [ ]:
# Import the IMDB dataset
imdb_df = pd.read_csv('IMDB Dataset.csv')
imdb_df.head()


,review,sentiment
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",positive


In [ ]:
# Change the name of the review column to "text" and the sentiment column to "label"
imdb_df.rename(columns={'review': 'text', 'sentiment': 'label'}, inplace=True)
imdb_df.head()

,text,label
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",positive
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",positive
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",positive
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",positive


In [ ]:
# Change the label column to have positive be 1 and negative be 0
imdb_df['label'] = imdb_df['label'].map({'positive': 1, 'negative': 0})
imdb_df.head()

,text,label
0,"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me...",1
1,"A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire p...",1
2,"I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue i...",1
3,Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenl...,0
4,"Petter Mattei's ""Love in the Time of Money"" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what mone...",1


In [ ]:
# Set the features variable to the "text" column
X = imdb_df['text']
# Set the target variable to the "label" column
y = imdb_df['label']
# Split the data into train, test, split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
custom_stopwords = ['a', 'about', 'an', 'and', 'are', 'as', 'at', 'be', 'been', 'but', 'by', 'can', \
             'even', 'ever', 'for', 'from', 'get', 'had', 'has', 'have', 'he', 'her', 'hers', 'his', \
             'how', 'i', 'if', 'in', 'into', 'is', 'it', 'its', 'just', 'me', 'my', 'of', 'on', 'or', \
             'see', 'seen', 'she', 'so', 'than', 'that', 'the', 'their', 'there', 'they', 'this', \
             'to', 'was', 'we', 'were', 'what', 'when', 'which', 'who', 'will', 'with', 'you']

In [ ]:
# Build a pipeline using TfidfVectorizer(), with custom_stopwords and LinearSVC
text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words=custom_stopwords, ngram_range=(1,2), max_df=0.9, min_df=2, sublinear_tf=True)),
                     ('clf', LinearSVC(C=1, loss='squared_hinge', penalty='l2', dual=True))])

# Fit the data to the model
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2),
                                 stop_words=['a', 'about', 'an', 'and', 'are',
                                             'as', 'at', 'be', 'been', 'but',
                                             'by', 'can', 'even', 'ever', 'for',
                                             'from', 'get', 'had', 'has',
                                             'have', 'he', 'her', 'hers', 'his',
                                             'how', 'i', 'if', 'in', 'into',
                                             'is', ...],
                                 sublinear_tf=True)),
                ('clf', LinearSVC(C=1, dual=True))])

In [ ]:
# Validate the model by checking the model accuracy with model.score
print('Train accuracy:', text_clf.score(X_train, y_train))
print('Test accuracy:', text_clf.score(X_test, y_test))

Train accuracy: 0.9998
Test accuracy: 0.9171


In [ ]:
# Create a confusion matrix on the test data and predictions
predictions = text_clf.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
print(metrics.classification_report(y_test,predictions))
print(metrics.accuracy_score(y_test,predictions))

[[4508  453]
 [ 376 4663]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      4961
           1       0.91      0.93      0.92      5039

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000

0.9171


In [ ]:
# Test a sample review "I thought the movie was slow and I wasn't very engaged"
sample = text_clf.predict(["I thought the movie was slow and I wasn't very engaged"])
sample

array([0])

In [ ]:
# Test another sample review "This movie brought me so much joy. I loved the part with the animals"
sample = text_clf.predict(["This movie brought me so much joy. I loved the part with the animals"])
sample

array([1])

In [ ]:
# Save the model
vectorizer = text_clf.named_steps['tfidf']
joblib.dump(vectorizer, 'vectorizer.pkl')
model = text_clf.named_steps['clf']
joblib.dump(model, 'model.pkl')

['model.pkl']

In [ ]:
#Load the saved model
new_vectorizer = joblib.load('vectorizer.pkl')
new_model = joblib.load('model.pkl')

In [ ]:
# Load the rt dataset
rt = load_dataset("cornell-movie-review-data/rotten_tomatoes")
# Extract the train dataset from the rt
rt_df = rt['train'].to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
rt_df.head()

NameError: name 'rt_df' is not defined

In [ ]:
rt_df.info()

In [ ]:
# Assuming the text column is named 'review'
new_texts = rt_df['text'].astype(str)

# Transform using the existing vectorizer
new_features = new_vectorizer.transform(new_texts)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Get actual labels
y_true = rt_df['label']

# Predict using the trained model
y_pred = new_model.predict(new_features)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Detailed evaluation
print(classification_report(y_true, y_pred))


In [ ]:

# Load your vectorizer and transform the dataset
vectorizer = joblib.load('vectorizer.pkl')
X = vectorizer.transform(rt_df['text'])  # Text features
y = rt_df['label']  # Target labels

# Initialize LinearSVC model
model = LinearSVC(C=1, loss='squared_hinge', penalty='l2', dual=True)

# Fit the model
model.fit(X, y)
# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

# Output the accuracy scores for each fold
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean accuracy: {cv_scores.mean()} ± {cv_scores.std()}")


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Define pipeline
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=custom_stopwords)),
    ('clf', LinearSVC())
])

# Define hyperparameter grid
param_grid = {
    'tfidf__ngram_range': [(1,1), (1,2), (1,3)],  # Unigrams, bigrams, trigrams
    'tfidf__max_df': [0.75, 0.9],  # Ignore too common words
    'tfidf__min_df': [2, 5],  # Ignore rare words
    'tfidf__sublinear_tf': [True, False],
    'clf__C': [0.1, 1, 10],  # Regularization
    'clf__loss': ['hinge', 'squared_hinge'],
    'clf__penalty': ['l2'],  # 'l1' only works with dual=False
    'clf__dual': [True, False]
}

# Run Grid Search
grid_search = GridSearchCV(text_clf, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best parameters and accuracy
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


KeyboardInterrupt: 

In [ ]:
#Download Vader package
nltk.download('vader_lexicon')

In [ ]:
# Better gradio interface


# Sentiment Analysis Function returning sentiment and score
def vader_review(text):
    analyzer = SentimentIntensityAnalyzer()
    score = analyzer.polarity_scores(text)['compound']
    if score >= 0.05:
        return "😊 Positive", score
    elif score <= -0.05:
        return "😠 Negative", score
    else:
        return "😐 Neutral", score

# Improved Gradio Interface
with gr.Blocks(css=".gradio-container {background-color: #ADD8E6; font-family: Arial;}") as demo:
    gr.Markdown("## 🌟 VADER 🌟")
    gr.Markdown("Enter your review, and we'll analyze its sentiment!")

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Enter your review",
            lines=3,
            max_lines=5,
            interactive=True,
            placeholder="Type your review here..."
        )

    analyze_button = gr.Button("🔍 Analyze Sentiment")

    with gr.Row():
        output_label = gr.Textbox(label="🔽 Sentiment Result", interactive=False)

    analyze_button.click(vader_review, input_text, output_label)

demo.launch()


In [ ]:
# Sentiment Analysis Function returning sentiment and score using text_clf
def mj_review(text):
    score = text_clf.predict([text])[0]
    if score == 1:
        return "😊 Positive"
    elif score == 0:
        return "😠 Negative"

# Improved Gradio Interface
with gr.Blocks(css=".gradio-container {background-color: #f5f7fa; font-family: Arial;}") as demo:
    gr.Markdown("## 🌟 Sentiment Analyzer 🌟")
    gr.Markdown("Enter your review, and we'll analyze its sentiment!")

    with gr.Row():
        input_text = gr.Textbox(
            label="📝 Enter your review",
            lines=3,
            max_lines=5,
            interactive=True,
            placeholder="Type your review here..."
        )

    analyze_button = gr.Button("🔍 Analyze Sentiment")

    with gr.Row():
        output_label = gr.Textbox(label="🔽 Sentiment Result", interactive=False)

    analyze_button.click(mj_review, input_text, output_label)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8284b2c87885830dd6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Initialize VADER
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment as only Positive or Negative
def get_vader_sentiment(text):
    score = analyzer.polarity_scores(str(text))["compound"]
    # Return 1 for positive and 0 for negative to match y_true
    return 1 if score >= 0 else 0

# Apply classification to the "text" column, not the "label" column
imdb_df["predicted_label"] = imdb_df["text"].apply(get_vader_sentiment)

# Print classification report (assuming 'actual_label' exists)
print(classification_report(imdb_df["label"], imdb_df["predicted_label"]))